# IMPORTS

In [1]:
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
import pickle
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

# WRITING HTML FILE

In [15]:
%%writefile templates/index.html

<!DOCTYPE html>  
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <title>NYC Taxi Trip Duration Prediction</title>
    <style>
    table{
        margin-left: "auto";
        margin-right: "auto";
    };
    input{
        width: 100%;
    }
    </style>
  </head>
  <body>
    <center>
    <h1>NYC Taxi Trip Duration Prediction</h1>
    <form action="{{url_for('predict')}}" method="POST">
      <table>
        <tr>
          <td>Pickup Latitude</td>
            <td>Pickup Longitude</td>
        </tr>
        <tr>
          <td>
            <input name="Pickup Latitude" size="25" maxlength="25" type="number" step="0.00000000000001"
            min = "40.69"
            max = "40.81"
            />
            </td>
            <td>
            <input name="Pickup Longitude" size="25" maxlength="25" type="number" step="0.00000000000001"
            min = "-74.02"
            max = "-73.93"
            />
            </td>
        </tr>
        
        <tr>
          <td>Dropoff Latitude</td>
            <td>Dropoff Longitude</td>
        </tr>
        <tr>
          <td>
            <input name="Dropoff Latitude" size="25" maxlength="25" type="number" step="0.00000000000001"
            min = "40.69"
            max = "40.81"
            />
            </td>
            <td>
            <input name="Dropoff Longitude" size="25" maxlength="25" type="number" step="0.00000000000001"
            min = "-74.03"
            max = "-73.92"
            />
            </td>
        </tr>
        <tr>
          <td colspan=2>
            <center>Date Time</br>
            <input name="Date Time" type="datetime-local"/></center>
            </td>
        </tr> 
        </table>
        <input type="submit" value="submit"
        style="border: 0px; width: 90px; height: 40px; background: blue; color: white; font-size: 16px; margin-top: 30px;"/>
    </form>
    <br />
    <h3>{{prediction_text}}</h3>
    </center>
    </body>
</html>

Overwriting templates/index.html


# FLASK

In [16]:
app = Flask(__name__)
run_with_ngrok(app)

# LOADING PRETRAINED MODEL
cb_model = CatBoostRegressor(iterations=2000,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 23,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100,
                             task_type="GPU",
                             devices='0:1'
                            )
cb_model.load_model('cb_model')

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    features = [[]]
    features[0].append(float(request.form['Pickup Longitude']))
    features[0].append(float(request.form['Pickup Latitude']))
    features[0].append(float(request.form['Dropoff Longitude']))
    features[0].append(float(request.form['Dropoff Latitude']))
    
    date = " ".join(request.form['Date Time'].split("T")) + ":00"
    date = pd.to_datetime(date)
    
    features[0].append(date.hour)
    
    dlon = float(request.form['Pickup Longitude']) - float(request.form['Dropoff Longitude'])
    dlat = float(request.form['Pickup Latitude']) - float(request.form['Dropoff Latitude'])
    a = (np.sin(dlat/2))**2 + np.cos(float(request.form['Pickup Latitude'])) * np.cos(float(request.form['Dropoff Latitude'])) * (np.sin(dlon/2))**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    features[0].append(6373.0 * c)
    
    
    features[0].append(date.weekofyear)
    features[0].append(date.minute)
    features[0].append(date.weekday() * 24 + date.hour)
    
    # PREDICTION FROM SAVED MODEL
    result = cb_model.predict(features)

    # SHOWING RESULT TO USER
    return render_template(
        "index.html",
        prediction_text="Trip Duration = {:.2f} Min".format(
            result[0]/60
        ),
    )

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Aug/2021 15:05:44] "GET / HTTP/1.1" 200 -


 * Running on http://7c73-103-226-144-197.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Aug/2021 15:06:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2021 15:06:52] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2021 15:08:39] "POST /predict HTTP/1.1" 200 -
